In [290]:
import pandas as pd
import numpy as np
import _pickle as pickle
import datetime
import random
import timestring

from bokeh.io import output_file, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.palettes import *
from bokeh.plotting import figure

fileHandler = open(b"../vagrant_shared_folder/movieRatings.obj", "rb")
movieRatings = pickle.load(fileHandler)
fileHandler.close()

output_notebook()

Loading BokehJS ...

In [291]:
def random_HEX_colors(n=1):

    color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(n)]
    
    return color

In [292]:
print(f'There are {len(movieRatings)} movies.')

There are 273 movies.


In [293]:
for ID, data in movieRatings.items():
    if 'Silence' in data['title']:
        print("{}, vote = {}".format(data['title'],data['personal_rating']))

The Silence of the Lambs, vote = 9


In [294]:
movieRatings['tt4550098']

{'ID': 'tt4550098',
 'type': 'movie',
 'title': 'Nocturnal Animals',
 'genre': 'Drama, Thriller',
 'year': '2016',
 'personal_rating': '9',
 'IMDB_rating': '7.5',
 'rated_on': '03 Jan 2021',
 'actors': ['Amy Adams',
  'Jake Gyllenhaal',
  'Michael Shannon',
  'Aaron Taylor-Johnson'],
 'director': 'Tom Ford'}

## Best-rated {9-10}

In [295]:
tens = []
nines = []

for ID, data in movieRatings.items():
    t = int(data['personal_rating'])
    if t in [9,10]:
        if t == 9:
            nines.append(data['title'])
        else:
            tens.append(data['title'])

In [296]:
print(f"10s: {tens}; {len(tens)}")
print(f"9s: {nines}; {len(nines)}")

10s: ['Soul', 'Heat', 'Se7en', 'The Last Dance', 'Ozark', 'When Harry Met Sally', 'Succession', 'Peaky Blinders', 'Game of Thrones', 'The Big Bang Theory']; 10
9s: ['The Night Of', 'Promising Young Woman', 'Dark Waters', 'Night Stalker: The Hunt for a Serial Killer', 'Run', 'Inside Out', 'Nocturnal Animals', 'The Boys', 'Prisoners', 'There Will Be Blood', '1917', '12 Angry Men', 'Casino', 'Nightcrawler', 'The Shawshank Redemption', 'Goodfellas', 'Inglourious Basterds', 'The Lobster', 'Moneyball', 'Apollo 11', 'Joker', 'BoJack Horseman', 'James May: Our Man in Japan', 'The Mandalorian', 'Once Upon a Time in America', 'The Day After Tomorrow', 'Mindhunter', 'Billions', 'Chernobyl', 'Presumed Innocent', 'Luther', 'Black Mirror: Bandersnatch', 'The Staircase', 'Making a Murderer', 'Black Mirror', 'Icarus', 'The Silence of the Lambs', 'Spotlight', 'Last Week Tonight with John Oliver', 'Halt and Catch Fire', 'Inception', 'The Godfather: Part II', 'The Godfather', 'House of Cards', 'Master of

## Year-based stats

In [297]:
def retrieve_year_specific_data(year, debug=False):
    data_year = []
    movies_year = []
    tv_year = []
    
    year = str(year)

    for ID, data in movieRatings.items():
        t = data['rated_on']
        if year in t:
            data_year.append(data)

    for d in data_year:
        t = d['type']
        if 'movie' in t:
            movies_year.append(d)
        else:
            tv_year.append(d)
    
    if len(movies_year)+len(tv_year) == len(data_year):
        if debug:
            print(f"Amount of movies/tv_series seen in {year}: {len(data_year)}")
            print(f"Movies seen in {year}: {len(movies_year)}")
            print(f"TV Series seen in {year}: {len(tv_year)}")
        return movies_year, tv_year
    else:
        print("Error...")
        return None, None

In [298]:
mov, tv = retrieve_year_specific_data(2021)
len(mov), len(tv)

(16, 6)

## Yearly-stats distribution

In [299]:
def yearly_stats_distribution():
    current_year = datetime.datetime.now().year + 1
    possible_years = [i for i in range(2014, current_year)]
    counts_tot = []
    counts_mov = []
    counts_tv = []
    
    effective_years = []

    for y in possible_years:
        mov, tv = retrieve_year_specific_data(y)

        effective_years.append(y)
        counts_tot.append(len(mov)+len(tv))
        counts_mov.append(len(mov))
        counts_tv.append(len(tv))

    strYears = []
    for _ in effective_years:
        strYears.append(str(_))
    
    return strYears, counts_tot, counts_mov, counts_tv

In [300]:
# output_file("bar_colors.html")

years, counts, _, _ = yearly_stats_distribution()

source = ColumnDataSource(data=dict(year=years, counts=counts, color=small_palettes['Viridis'][len(years)]))

p = figure(x_range=years, y_range=(min(counts),max(counts)+20), plot_height=350, title="Yearly Distribution",
           toolbar_location=None, tools="hover", tooltips="data points: @counts")

p.vbar(x='year', top='counts', width=0.9, color='color', source=source)

p.xgrid.grid_line_color = None

show(p)

In [301]:
years, counts, mov, tv = yearly_stats_distribution()

type_data = {
    'years'    : years,
    'movies'   : mov,
    'tv series': tv
            }

p = figure(x_range=years, plot_height=250,y_range=(min(counts),max(counts)+20), title="Yearly Distribution by Type",
           toolbar_location=None, tools="hover", tooltips="$name in @years: @$name")

p.vbar_stack(['movies', 'tv series'], x='years', width=0.9, color=["#FFC300", "#058C0B"], source=type_data,
             legend_label=['movies', 'tv series'])

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"

show(p)

## Genre-based analysis

In [302]:
def build_genre_dict():
    genres = {}

    for ID, data in movieRatings.items():
        genre_current = data['genre'].split(',')
        for g in genre_current:
            g = g.strip()
            try:
                genres[g] += 1
            except KeyError:
                genres[g] = 1
            
    return genres

In [303]:
genre_dict = build_genre_dict()
genre_dict

{'Crime': 108,
 'Drama': 184,
 'Mystery': 54,
 'Thriller': 77,
 'Adventure': 34,
 'Comedy': 52,
 'Fantasy': 11,
 'Biography': 29,
 'History': 20,
 'Documentary': 24,
 'Horror': 20,
 'Sci-Fi': 26,
 'Action': 46,
 'Animation': 5,
 'Romance': 18,
 'Family': 2,
 'War': 7,
 'Sport': 5,
 'News': 1,
 'Talk-Show': 2,
 'Music': 2}

In [304]:
# Genre-based on yearly data
def genre_yearly_data():
    current_year = datetime.datetime.now().year + 1
    possible_years = [str(i) for i in range(2014, current_year)]

    current_year_mov, current_year_tv = None, None
    yearly_genres = {}
    
    for year in possible_years:
        
        yearly_genres[year] = {}
        
        current_year_mov, current_year_tv = retrieve_year_specific_data(year)
        current_year_data = current_year_mov + current_year_tv
        
        for data in current_year_data:
            for g in data['genre'].split(','):
                g = g.strip()
                try:
                    yearly_genres[year][g] += 1
                except KeyError:
                    yearly_genres[year][g] = 1
                    
    return yearly_genres

In [305]:
genre_yearly_data()

{'2014': {},
 '2015': {'Crime': 4,
  'Mystery': 1,
  'Thriller': 6,
  'Comedy': 6,
  'Romance': 5,
  'Action': 4,
  'Adventure': 3,
  'Sci-Fi': 2,
  'Biography': 2,
  'Drama': 11,
  'Music': 1,
  'History': 1},
 '2016': {'Crime': 9,
  'Drama': 27,
  'Mystery': 8,
  'Action': 7,
  'Adventure': 5,
  'Fantasy': 4,
  'Thriller': 3,
  'Comedy': 9,
  'Romance': 2,
  'Sci-Fi': 2,
  'Biography': 8,
  'Horror': 3,
  'Documentary': 2,
  'History': 3,
  'War': 3,
  'Talk-Show': 1,
  'Music': 1,
  'Animation': 1},
 '2017': {'Drama': 30,
  'Sci-Fi': 6,
  'Thriller': 13,
  'Crime': 16,
  'Romance': 4,
  'Biography': 7,
  'Action': 11,
  'Adventure': 6,
  'Comedy': 5,
  'Horror': 4,
  'Documentary': 2,
  'History': 4,
  'Fantasy': 2,
  'Mystery': 7,
  'War': 1,
  'News': 1,
  'Talk-Show': 1},
 '2018': {'Drama': 19,
  'Mystery': 8,
  'Thriller': 13,
  'Action': 9,
  'Adventure': 6,
  'Sci-Fi': 6,
  'Crime': 12,
  'Biography': 2,
  'Sport': 2,
  'History': 2,
  'Comedy': 2,
  'Horror': 2,
  'Documentar

In [306]:
def build_data_source_for_graph():
    all_possible_genres_seen = build_genre_dict().keys()

    source = {}

    yearly_data = genre_yearly_data()

    source['years'] = [str(i) for i in yearly_data.keys()]

    for genre in all_possible_genres_seen:
        source[genre] = [0 for i in range(0, len(source['years']))]

    for n, year in enumerate(yearly_data.keys()):
        genres = yearly_data[year]
        for g in genres:
            source[g][n] = genres[g]

    return all_possible_genres_seen, source

In [307]:
all_possible_genres_seen, source = build_data_source_for_graph()

labels = [str(l) for l in all_possible_genres_seen]

p = figure(x_range=source['years'], plot_height=450, plot_width=1000, y_range=(0,200),
           title="Genre Distribution per Year",
           toolbar_location=None, tools="hover", tooltips="$name in @years: @$name")

p.vbar_stack(labels, x='years', width=0.3, 
             color=random_HEX_colors(len(labels)), source=source,
             legend_label=labels)

# p.y_range.start = 0
p.x_range.range_padding = .15
# p.min_border_bottom=10
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None

p.legend.orientation = "horizontal"
# p.legend.label_text_font_size = "5px"

new_legend = p.legend[0]
p.add_layout(new_legend, 'below')

show(p)

## Most recent data

In [308]:
def retrieve_most_recent_seen():
    
    print("""Retrieving the most recently seen shows/movies this past 2 weeks""")
    
    recent_data = []
    
    last_2_weeks = timestring.Range('last 14 days')
    
    for ID, data in movieRatings.items():
        d = data['rated_on'].split(' ')
        finalDate = d[1] +' ' + d[0] + ' ' + d[2]
        finalDate = timestring.Date(finalDate)
        
        if finalDate not in last_2_weeks:
            break
        else:
            recent_data.append(data)
        
    return recent_data


retrieve_most_recent_seen()

Retrieving the most recently seen shows/movies this past 2 weeks


[{'ID': 'tt2401256',
  'type': 'tv_series',
  'title': 'The Night Of',
  'genre': 'Crime, Drama, Mystery',
  'year': '2016',
  'personal_rating': '9',
  'IMDB_rating': '8.5',
  'rated_on': '11 Feb 2021',
  'actors': ['Riz Ahmed', 'John Turturro', 'Bill Camp', 'Amara Karan'],
  'director': ''},
 {'ID': 'tt9620292',
  'type': 'movie',
  'title': 'Promising Young Woman',
  'genre': 'Crime, Drama, Thriller',
  'year': '2020',
  'personal_rating': '9',
  'IMDB_rating': '7.5',
  'rated_on': '08 Feb 2021',
  'actors': ['Carey Mulligan', 'Bo Burnham', 'Alison Brie', 'Clancy Brown'],
  'director': 'Emerald Fennell'},
 {'ID': 'tt10016180',
  'type': 'movie',
  'title': 'The Little Things',
  'genre': 'Crime, Drama, Thriller',
  'year': '2021',
  'personal_rating': '6',
  'IMDB_rating': '6.3',
  'rated_on': '08 Feb 2021',
  'actors': ['Denzel Washington', 'Rami Malek', 'Jared Leto', 'Chris Bauer'],
  'director': 'John Lee Hancock'},
 {'ID': 'tt10287954',
  'type': 'tv_series',
  'title': 'Rose Is

## TMDB API

In [309]:
import requests
import json
import shutil
import glob, os
from PIL import Image

TMDB_API = '2410c5007d1bd89e6f524c8659332ca4'
external_source = '&external_source=imdb_id'
img_TMDB = ''
img_url = []
        
recent_data = retrieve_most_recent_seen()

for recent in recent_data:
    ID_find = recent['ID']
    r = requests.get(f'https://api.themoviedb.org/3/find/{ID_find}?api_key={TMDB_API}&{external_source}')
    data = r.json()

    for i in data:
        for data_inner in data[i]:
            if data_inner != '':
                img_TMDB = data_inner['poster_path']
                
    img_url.append(f'https://image.tmdb.org/t/p/w500{img_TMDB}')

    
for n, img in enumerate(img_url):
    response = requests.get(img, stream=True)
    with open(f'img_{n}.png', 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response

Retrieving the most recently seen shows/movies this past 2 weeks


## Retrieve people data

In [310]:
a = movieRatings['tt0887883']['director']

# t = []

# for act in a:
#     t.append(act)

# occ = {}
# # Counting occurrences
# for _ in t:
#     try:
#         occ[_] += 1
#     except KeyError:
#         occ[_] = 1
        
# occ

a

'Ethan Coen, Joel Coen'

In [317]:
def retrieve_people(d):
    actors = {}
    directors = {}
    
    for ID, data in d.items():
        tmp_act = data['actors']
        tmp_dir = data['director']
        
        if tmp_act != '':
            for _ in tmp_act:
                try:
                    if _ == '':
                        print(f"This: {data['title']}")
                    actors[_] += 1
                except KeyError:
                    actors[_] = 1
                    
        # TODO: Work on the directors!                    
        if tmp_dir != '':
            if not str.isupper(tmp_dir[0]):
                print(data)
            try:
                directors[tmp_dir] += 1
            except KeyError:
                directors[tmp_dir] = 1
    
    return directors, actors

di, ac = retrieve_people(movieRatings)

In [332]:
def bulk_download_images(people_type, url):
    
    for n, img in enumerate(url):
        response = requests.get(img, stream=True)
        with open(f'{people_type}_{n}.png', 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        del response
        
    return f"Downloaded {people_type}-related data."

In [323]:
sorted_actors = dict(sorted(ac.items(), key=lambda item: item[1]))
sorted_directors = dict(sorted(di.items(), key=lambda item: item[1]))

top_three_actors = {k: sorted_actors[k] for k in list(sorted_actors)[len(sorted_actors)-3:]}
top_three_directors = {k: sorted_directors[k] for k in list(sorted_directors)[len(sorted_directors)-3:]}
top_three_actors, top_three_directors

({'Brad Pitt': 7, 'Jake Gyllenhaal': 8, 'Robert De Niro': 14},
 {'Christopher Nolan': 4, 'David Fincher': 5, 'Martin Scorsese': 7})

In [333]:
# Retriving TMDB images for actors
TMDB_API = '2410c5007d1bd89e6f524c8659332ca4'
img_TMDB = ''
img_url = []
        
# https://api.themoviedb.org/3/search/person?api_key=2410c5007d1bd89e6f524c8659332ca4&query=asdasd

for actor in top_three_actors.keys():
    query = actor
    page_number = 1
    r = requests.get(f'https://api.themoviedb.org/3/search/person?api_key={TMDB_API}&query={query}&page={page_number}')
    data = r.json()

    for d in data['results']:
        img_TMDB = d['profile_path']
        if img_TMDB != None:
            img_url.append(f'https://image.tmdb.org/t/p/w500{img_TMDB}')
            break


bulk_download_images('actor', img_url)

'Downloaded actor-related data.'

In [334]:
# Retriving TMDB images for directors
TMDB_API = '2410c5007d1bd89e6f524c8659332ca4'
img_TMDB = ''
img_url = []
        
# https://api.themoviedb.org/3/search/person?api_key=2410c5007d1bd89e6f524c8659332ca4&query=asdasd

for director in top_three_directors.keys():
    query = director
    page_number = 1
    r = requests.get(f'https://api.themoviedb.org/3/search/person?api_key={TMDB_API}&query={query}&page={page_number}')
    data = r.json()

    for d in data['results']:
        img_TMDB = d['profile_path']
        if img_TMDB != None:
            img_url.append(f'https://image.tmdb.org/t/p/w500{img_TMDB}')
            break

bulk_download_images('director', img_url)

'Downloaded director-related data.'